In [15]:
import pyomo.environ as pyo
import numpy as np
from matplotlib import pyplot as plt
import math
import scipy.stats as stats
import pandas as pd

In [16]:
data = pd.read_csv("NaCl_CaCl2.csv")
NaCl_conc = data["NaCl conc"]
CaCl2_conc = data["CaCl2 conc"]
cond = data["Bulk cond"]
ndata = len(cond)

number_density_NaCl = [NaCl_conc[i]*(6.022*10**23) for i in range(ndata)]
number_density_CaCl2 = [CaCl2_conc[i]*(6.022*10**23) for i in range(ndata)]

In [17]:
# k_B = 1.381 * 10**(-23) * pyo.units.J / pyo.units.K
# T = 298.15 * pyo.units.K
# charge = 1.602 * 10**(-19) * pyo.units.C
# D_Na = 1.334 * 10**(-9) * (pyo.units.m)**2 / pyo.units.s
# D_Cl = 2.032 * 10**(-9) * (pyo.units.m)**2 / pyo.units.s
# D_Ca = 0.792 * 10**(-9) * (pyo.units.m)**2 / pyo.units.s
# E_field = 0.7 * pyo.units.V / pyo.units.m
# z_Na = 1
# z_Cl = -1
# z_Ca = 2
# epsilon = 78.4
# epsilon_0 = 8.854 * 10**(-12) * (pyo.units.C)**2 / (pyo.units.N * (pyo.units.m)**2)
# sigma_Na = 100 * 10**(-12) * pyo.units.m
# sigma_Cl = 190 * 10**(-12) * pyo.units.m
# sigma_Ca = 140 * 10**(-12) * pyo.units.m

In [20]:
k_B = 1.381 * 10**(-23)
T = 298.15
charge = 1.602 * 10**(-19)
D_Na = 1.334 * 10**(-9)
D_Cl = 2.032 * 10**(-9)
D_Ca = 0.792 * 10**(-9)
E_field = 0.7
z_Na = 1
z_Cl = -1
z_Ca = 2
epsilon = 78.4
epsilon_0 = 8.854 * 10**(-12)
sigma_Na = 100 * 10**(-12)
sigma_Cl = 190 * 10**(-12)
sigma_Ca = 140 * 10**(-12)

In [28]:
components = ["Na", "Ca", "Cl"]
valency = [z_Na, z_Ca, z_Cl]
diff_coeff = [D_Na, D_Ca, D_Cl]
number_density_Na = number_density_NaCl
number_density_Ca = number_density_CaCl2
number_density_Cl = number_density_NaCl + 2*number_density_CaCl2
# number_density_all = [number_density_Na, number_density_Ca, number_density_Cl]

for l in range(ndata):
    number_density = [number_density_Na[l], number_density_Ca[l], number_density_Cl[l]]
    omega = []
    mew = []
    for i in range(len(components)):
        # evaluating the velocity produced by the net electric force on species i
        vel = valency[i]*charge*E_field*diff_coeff[i]/(k_B*T)

        # evaluating relaxation correction
        ## evaluating omega
        omega_i = diff_coeff[i]/(k_B*T)
        omega.append(omega_i)

        ## evaluating kappa
        kappa_sq = 0.0
        for j in range(len(components)):
            kappa_j = number_density[j]*(valency[j]*charge)**2/(epsilon*epsilon_0*k_B*T)
            kappa_sq += kappa_j
        kappa = math.sqrt(kappa_sq)

        ## evaluating mew, the relative ionic strength
        mew_deno = 0.0
        for s in range(len(components)):
            mew_deno_s = number_density[s]*(valency[s]*charge)**2
            mew_deno += mew_deno_s
        mew_i = number_density[i]*(valency[i]*charge)**2/mew_deno
        mew.append(mew_i)

    # evaluating the mean mobility, omega_bar
    omega_bar = 0.0
    for r in range(len(components)):
        omega_bar_r = mew[r]*omega[r]
        omega_bar += omega_bar_r
        
            
            

C:\Users\lilon\AppData\Local\Temp\ipykernel_27768\1697994160.py:34: RuntimeWarning: invalid value encountered in scalar divide
  mew_i = number_density[i]*(valency[i]*charge)**2/mew_deno
